In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import binarize
import sklearn
from sklearn.svm import LinearSVC
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
import nltk
import os
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer

In [4]:
columns=['value','twitterhandle','userid','user_screen_name','retweets','favorites','tweet_id','timestamp','tweet']
#df=pd.read_csv('/home/ds/notebooks/Data/twitterTweets.csv',sep='\t',names=columns,encoding='utf-8')

In [2]:
def read_csv(name):
    location='/home/ds/notebooks/Data/'
    name=name+'.csv'
    columns=['value','twitterhandle','userid','user_screen_name','retweets','favorites','tweet_id','timestamp','tweet']
    for root,dirs,files in os.walk(location):
        if name in files:
            outfile=os.path.join(root,name)
            return pd.read_csv(outfile,sep='\t',names=columns,encoding='utf-8')#,usecols=['value', 'tweet'])
            
def training(df):
    return np.random.rand(len(df)) < 0.8
#df_train=df[msk]
#df_test=df[~msk]
#return df_train.shape,df_test.shape
#def 

In [3]:
df = read_csv('twitterTweets')

/opt/ds/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
df_train=df[training(df)]
df_test=df[~training(df)]

In [5]:
df2=pd.read_csv('/home/ds/notebooks/Data/twitterhandles.csv',encoding='utf-8')
df2.rename(columns={'Handle': 'value'}, inplace=True)
df_train=pd.merge(df_train, df2, on='value', how='inner')
df_test=pd.merge(df_test, df2, on='value', how='inner')
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
df_train['age_group'] = pd.cut(df_train.Age, range(0, 81, 10), right=False, labels=labels)
df_test['age_group'] = pd.cut(df_test.Age, range(0, 81, 10), right=False, labels=labels)
df_train=df_train.dropna()
df_test=df_test.dropna()

In [6]:
%timeit 
df3.isnull().sum()

NameError: name 'df3' is not defined

In [53]:
def vectorize(df):
    bow_transformer = CountVectorizer(tokenizer=nltk.word_tokenize,token_pattern="[a-zA-Z]{2,15}",stop_words="english")
    bow_transformer.fit(df)
    messages_bow = bow_transformer.transform(df)
    tfidf_transformer = TfidfTransformer().fit(messages_bow)
    messages_tfidf = tfidf_transformer.transform(messages_bow)
    return messages_tfidf    

In [29]:
%timeit age_inference = MultinomialNB().fit(vectorize(df_train['tweet']), df_train['age_group'])

1 loops, best of 3: 2min 29s per loop


In [31]:
bow_transformer = CountVectorizer(tokenizer=nltk.word_tokenize,token_pattern="[a-zA-Z]{2,15}",stop_words="english"
                                 ).fit(df_train['tweet'])

In [32]:
message3 = df_train['tweet'][3]
print message3
bow4 = bow_transformer.transform([message3])
print bow4
print bow4.shape
print bow_transformer.get_feature_names()[129168]
print bow_transformer.get_feature_names()[115477]

@centrixe u good ruben
  (0, 29594)	1
  (0, 43505)	1
  (0, 62416)	1
  (0, 107909)	1
  (0, 123541)	1
(1, 135066)
wizkalifah
stillness


In [34]:
messages_bow = bow_transformer.transform(df_train['tweet'])
print 'sparse matrix shape:', messages_bow.shape
print 'number of non-zeros:', messages_bow.nnz
print 'sparsity: %.2f%%' % (100.0 * messages_bow.nnz / (messages_bow.shape[0] * messages_bow.shape[1]))

sparse matrix shape: (181758, 135066)
number of non-zeros: 1485198
sparsity: 0.01%


In [35]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(messages_bow)
tfidf4 = tfidf_transformer.transform(bow4)
print tfidf4

  (0, 123541)	0.270434265522
  (0, 107909)	0.710601986685
  (0, 62416)	0.296011673885
  (0, 43505)	0.571041969021
  (0, 29594)	0.0905443708425


In [43]:
messages_tfidf = tfidf_transformer.transform(messages_bow)
print messages_tfidf.shape

(181758, 135066)


In [8]:
from sklearn.naive_bayes import MultinomialNB
#%time age_inference = MultinomialNB().fit(messages_tfidf, df_train['age_group'])

In [36]:
print 'predicted:', age_inference.predict(tfidf4)[:1]
print 'expected:', df_train['age_group'][:1]

predicted: ['10-19']
expected: 0    10-19
Name: age_group, dtype: category
Categories (8, object): [0-9 < 10-19 < 20-29 < 30-39 < 40-49 < 50-59 < 60-69 < 70-79]


In [15]:
all_predictions = age_inference.predict(messages_tfidf)
print all_predictions[:10]

['10-19' '10-19' '10-19' '10-19' '10-19' '10-19' '10-19' '10-19' '10-19'
 '10-19']


In [16]:
pd.DataFrame(all_predictions).groupby(0).size()

0
0-9         785
10-19    135469
20-29     10939
30-39      8063
40-49     25037
50-59      1055
60-69       324
70-79         5
dtype: int64

In [40]:
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score,confusion_matrix

In [18]:
print 'accuracy', accuracy_score(df_train['age_group'], all_predictions)
print 'confusion matrix\n', confusion_matrix(df_train['age_group'], all_predictions)
print '(row=expected, col=predicted)'

accuracy 0.817731468485
confusion matrix
[[   785    548      8      1     63      0      0      0]
 [     0 103421      3      1     41      0      0      0]
 [     0  10081  10905      2     72      0      0      0]
 [     0   7554      4   8020    341      0      0      0]
 [     0   7179      3      3  24050      0      0      0]
 [     0   3690     12     10    186   1053      0      0]
 [     0   2399      3     17    223      2    324      0]
 [     0    597      1      9     61      0      0      5]]
(row=expected, col=predicted)


In [37]:
messages_bow_test=bow_transformer.transform(df_test['tweet'])
messages_tfidf_test=tfidf_transformer.transform(messages_bow_test)

In [38]:
all_predictions_test=age_inference.predict(messages_tfidf_test)

In [41]:
print 'accuracy', accuracy_score(df_test['age_group'], all_predictions_test)
print 'confusion matrix\n', confusion_matrix(df_test['age_group'], all_predictions_test)
#print '(row=expected, col=predicted)'

accuracy 0.81137269584
confusion matrix
[[  183   171     2     0    13     0     0     0]
 [    0 25872     2     0    19     0     0     0]
 [    0  2586  2725     0    19     0     0     0]
 [    0  1958     0  1912    99     0     0     0]
 [    0  1911     0     1  5768     0     0     0]
 [    0   902     4     4    53   293     0     0]
 [    0   608     1     1    56     0    89     0]
 [    0   132     0     2    21     0     0     0]]


In [25]:
confusion=pd.DataFrame(confusion_matrix(df_test['age_group'], all_predictions_test))
confusion.index = ["%s (actual)" % x for x in labels]
confusion.columns = ["%s (predicted)" % x for x in labels] 
confusion

,0-9 (predicted),10-19 (predicted),20-29 (predicted),30-39 (predicted),40-49 (predicted),50-59 (predicted),60-69 (predicted),70-79 (predicted)
0-9 (actual),202,136,2,0,9,0,0,0
10-19 (actual),0,25681,0,0,26,0,0,0
20-29 (actual),0,2597,2658,0,19,0,0,0
30-39 (actual),0,1954,0,1866,77,0,0,0
40-49 (actual),0,2015,1,0,5828,0,0,0
50-59 (actual),0,899,4,1,51,253,0,0
60-69 (actual),0,628,0,7,58,0,65,0
70-79 (actual),0,142,0,2,15,0,0,1


In [60]:
pd.DataFrame(all_predictions_test).groupby(0).size()

0
0-9        127
10-19    35828
20-29     2212
30-39     1530
40-49     5423
50-59      245
60-69       55
dtype: int64

In [ ]:
rf = RandomForestRegressor()
rf.fit(X, Y)
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), 
             reverse=True)

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
df_train['Age_Encoded']=LabelEncoder().fit_transform(df_train['age_group']) 
#rf = RandomForestRegressor()
#RF=rf.fit(messages_tfidf, df_train['Age_Encoded'])

In [24]:
df_train.groupby('Age_Encoded').size()

Age_Encoded
0      1399
1    103531
2     20954
3     15918
4     31271
5      4915
6      2962
7       642
dtype: int64

In [ ]:
%time age_inference_features = RandomForestRegressor().fit(messages_tfidf, df_train['Age_Encoded'])

CPU times: user 43min 30s, sys: 192 ms, total: 43min 30s
Wall time: 43min 36s


In [ ]:
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), age_inference_features.feature_importances_), names), 
             reverse=True)

In [50]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=2)
random_forest=rf.fit(messages_tfidf,df_train['age_group'])
#.predict(messages_tfidf_test)

In [51]:
confusion=pd.DataFrame(confusion_matrix(df_test['age_group'], random_forest.predict(messages_tfidf_test)))
confusion.index = ["%s (actual)" % x for x in labels]
confusion.columns = ["%s (predicted)" % x for x in labels] 
confusion

,0-9 (predicted),10-19 (predicted),20-29 (predicted),30-39 (predicted),40-49 (predicted),50-59 (predicted),60-69 (predicted),70-79 (predicted)
0-9 (actual),365,4,0,0,0,0,0,0
10-19 (actual),0,25835,18,9,31,0,0,0
20-29 (actual),0,532,4782,4,12,0,0,0
30-39 (actual),1,212,13,3705,38,0,0,0
40-49 (actual),1,393,14,29,7243,0,0,0
50-59 (actual),0,119,2,12,13,1109,1,0
60-69 (actual),0,74,4,4,15,3,655,0
70-79 (actual),0,16,1,2,2,0,1,133


In [52]:
print 'accuracy', accuracy_score(df_test['age_group'],random_forest.predict(messages_tfidf_test))

accuracy 0.965203602969


In [54]:
x='I am now 25 years old and my eyesight is worsening, at what point do I get adult supervision? :nerd_face:'

In [60]:
import codecs
fo = codecs.open('/home/ds/notebooks/Data/tweet_test.txt', "r", "utf-8")
lines = fo.readlines()
fo.close()

In [67]:
for x in lines:
    tweet_test= bow_transformer.transform(x.strip())

In [70]:
test=age_inference.predict(tweet_test)

In [86]:
print 'tweet', lines[]
print 'predicted:', random_forest.predict(tweet_test)[3]

tweet My mom needs to chill.

predicted: 10-19


In [88]:
test

array(['10-19', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '10-19', '10-19', '10-19', '10-19', '10-19', '10-19',
       '10-19', '10-